### 1. Import packages

In [1]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

### 2. Helper functions

In [2]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    
    return state

### 3. Initialize the environment

In [3]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
#print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = len(env.observation) + 8
nb_actions = 200
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

27 9

Ram information received from DASSAT will has 35 dimensions.
There are 200 possible actions at each step.
Discrete? False


### 4. Define the network

In [4]:
class QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc2(x)

### 5. Define the Replay Buffer

In [5]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            how many samples stored in buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # store [s,a,r,s',done] for each sample in buffer
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # vstack is add rows together as a single matrix
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        #get all the states, actions, rewards, next_states, dones for all elements in this sample batch, each as a single matrix
        #device here is cpu?
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [6]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [7]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # add current (s,r,a,s') and sample a batch and learn if update_every
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        # return the action index with the largest value
        else:
            return random.choice(np.arange(self.action_size))
            #return one action randomly with possibility eps

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # detach for diemnsion correctness
        # GET largest value of self.qnetwork_target(next_states)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # 0 if done

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        #Why need to gather? actions are indexs?

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()
        #固定用法, change learning rate

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())

### DQN for nitrogen management

In [8]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions)

In [9]:
def dqn(n_episodes=2000, max_t=500, eps_start=1.0, eps_end=0.01, eps_decay=0.997, solved=495, optimize = True):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    start = time.time()                # Start time
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    list_eps = []
    eps = eps_start                    # initialize epsilon
    action_list=[]
    action_num_list=[]
    action_not_zero=0
    for i_episode in range(1, n_episodes+1):
        # Reset env and score at the beginning of episode
        action_not_zero=0
        state = env.reset() # reset the environment and get current state
        state = dict2array(state)
        score = 0 # initialize the score
        for t in range(max_t):
            action1 = agent.act(state, eps) if optimize else 100
            if action1 != 0:
                action_not_zero=action_not_zero+1
            if i_episode == 1000:
                action_list.append(action1)
            #action_n++
            # for debug
            action = {
                    'anfer': action1,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': 0,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
            next_state, reward, done, _ = env.step(action)      # send the action to the environment\
            if done:
                print('episode is %d .' % i_episode)
                print('action_not_zero is %d .' % action_not_zero)
                print('score is %d .' % score)
                print('')
                if i_episode > 900:
                    action_num_list.append(action_not_zero)
                break
            if action_not_zero > 8:
                if action1 > 0:
                    #print('satisfied')
                    reward = reward-1000*(action_not_zero-8)
            next_state = dict2array(next_state)
            #print('after reward is %d .' % reward)
            agent.step(state, action['anfer'], reward, next_state, done)
            state = next_state
            score += reward
            #append

        if i_episode == 1000:
            print('action_not_zero is %d .' % action_not_zero)
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        list_eps.append(eps)
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
            
        
        if i_episode % 100 == 0:
            print('\rEpisode {}/{} \tAverage Score: {:.2f}'.format(i_episode, n_episodes, np.mean(scores_window)))
            print('Epsilon: {}'.format(eps))
            
        if np.mean(scores_window)>solved:
            print('Game Solved after {} episodes'.format(i_episode))
            break

    time_elapsed = time.time() - start
    print("Time Elapse: {:.2f} seconds".format(time_elapsed))
    
    return scores, list_eps, action_list, action_num_list

In [10]:
scores, list_eps, action_list, action_num_list= dqn(n_episodes=1500)

episode is 1 .
action_not_zero is 155 .
score is -10741763 .

episode is 2 .
action_not_zero is 153 .
score is -10449821 .

episode is 3 .
action_not_zero is 153 .
score is -10450693 .

episode is 4 .
action_not_zero is 155 .
score is -10741341 .

episode is 5 .
action_not_zero is 154 .
score is -10595756 .

episode is 6 .
action_not_zero is 155 .
score is -10741437 .

episode is 7 .
action_not_zero is 155 .
score is -10741146 .

episode is 8 .
action_not_zero is 154 .
score is -10595246 .

episode is 9 .
action_not_zero is 152 .
score is -10306189 .

episode is 10 .
action_not_zero is 154 .
score is -10596524 .

episode is 11 .
action_not_zero is 154 .
score is -10594868 .

episode is 12 .
action_not_zero is 153 .
score is -10450649 .

episode is 13 .
action_not_zero is 152 .
score is -10306789 .

episode is 14 .
action_not_zero is 155 .
score is -10741742 .

episode is 15 .
action_not_zero is 155 .
score is -10741341 .

episode is 16 .
action_not_zero is 155 .
score is -10741447 .

e

episode is 132 .
action_not_zero is 114 .
score is -5573182 .

episode is 133 .
action_not_zero is 102 .
score is -4378931 .

episode is 134 .
action_not_zero is 92 .
score is -3492609 .

episode is 135 .
action_not_zero is 99 .
score is -4101705 .

episode is 136 .
action_not_zero is 100 .
score is -4192552 .

episode is 137 .
action_not_zero is 116 .
score is -5786567 .

episode is 138 .
action_not_zero is 90 .
score is -3408751 .

episode is 139 .
action_not_zero is 98 .
score is -4101691 .

episode is 140 .
action_not_zero is 109 .
score is -5057916 .

episode is 141 .
action_not_zero is 106 .
score is -4760615 .

episode is 142 .
action_not_zero is 101 .
score is -4377797 .

episode is 143 .
action_not_zero is 94 .
score is -3747121 .

episode is 144 .
action_not_zero is 102 .
score is -4378337 .

episode is 145 .
action_not_zero is 100 .
score is -4285012 .

episode is 146 .
action_not_zero is 91 .
score is -3409436 .

episode is 147 .
action_not_zero is 98 .
score is -4012725 .


episode is 263 .
action_not_zero is 74 .
score is -2216587 .

episode is 264 .
action_not_zero is 59 .
score is -1329924 .

episode is 265 .
action_not_zero is 69 .
score is -1895942 .

episode is 266 .
action_not_zero is 60 .
score is -1382640 .

episode is 267 .
action_not_zero is 79 .
score is -2561194 .

episode is 268 .
action_not_zero is 57 .
score is -1179917 .

episode is 269 .
action_not_zero is 72 .
score is -2021018 .

episode is 270 .
action_not_zero is 65 .
score is -1600981 .

episode is 271 .
action_not_zero is 78 .
score is -2490805 .

episode is 272 .
action_not_zero is 72 .
score is -2084667 .

episode is 273 .
action_not_zero is 60 .
score is -1330187 .

episode is 274 .
action_not_zero is 64 .
score is -1544445 .

episode is 275 .
action_not_zero is 70 .
score is -1896273 .

episode is 276 .
action_not_zero is 64 .
score is -1600986 .

episode is 277 .
action_not_zero is 77 .
score is -2421185 .

episode is 278 .
action_not_zero is 69 .
score is -1897082 .

episode 

episode is 395 .
action_not_zero is 44 .
score is -669636 .

episode is 396 .
action_not_zero is 43 .
score is -598266 .

episode is 397 .
action_not_zero is 65 .
score is -1600799 .

episode is 398 .
action_not_zero is 54 .
score is -1084907 .

episode is 399 .
action_not_zero is 40 .
score is -498575 .

episode is 400 .
action_not_zero is 56 .
score is -1180024 .

Episode 400/1500 	Average Score: -1066105.61
Epsilon: 0.30065146518438607
episode is 401 .
action_not_zero is 46 .
score is -744642 .

episode is 402 .
action_not_zero is 57 .
score is -1229617 .

episode is 403 .
action_not_zero is 43 .
score is -598388 .

episode is 404 .
action_not_zero is 49 .
score is -864548 .

episode is 405 .
action_not_zero is 48 .
score is -823363 .

episode is 406 .
action_not_zero is 47 .
score is -744346 .

episode is 407 .
action_not_zero is 41 .
score is -563930 .

episode is 408 .
action_not_zero is 47 .
score is -783373 .

episode is 409 .
action_not_zero is 40 .
score is -530496 .

episode

episode is 527 .
action_not_zero is 29 .
score is -233078 .

episode is 528 .
action_not_zero is 27 .
score is -192088 .

episode is 529 .
action_not_zero is 31 .
score is -278472 .

episode is 530 .
action_not_zero is 27 .
score is -173310 .

episode is 531 .
action_not_zero is 37 .
score is -408999 .

episode is 532 .
action_not_zero is 30 .
score is -255195 .

episode is 533 .
action_not_zero is 30 .
score is -255518 .

episode is 534 .
action_not_zero is 32 .
score is -277949 .

episode is 535 .
action_not_zero is 32 .
score is -302210 .

episode is 536 .
action_not_zero is 31 .
score is -278949 .

episode is 537 .
action_not_zero is 35 .
score is -381098 .

episode is 538 .
action_not_zero is 41 .
score is -531267 .

episode is 539 .
action_not_zero is 35 .
score is -381525 .

episode is 540 .
action_not_zero is 37 .
score is -437533 .

episode is 541 .
action_not_zero is 28 .
score is -212528 .

episode is 542 .
action_not_zero is 26 .
score is -155253 .

episode is 543 .
action_

...

episode is 934 .
action_not_zero is 13 .
score is -11108 .

episode is 935 .
action_not_zero is 12 .
score is -11403 .

episode is 936 .
action_not_zero is 8 .
score is -975 .

episode is 937 .
action_not_zero is 9 .
score is -2106 .

episode is 938 .
action_not_zero is 10 .
score is -3972 .

episode is 939 .
action_not_zero is 7 .
score is -968 .

episode is 940 .
action_not_zero is 12 .
score is -11131 .

episode is 941 .
action_not_zero is 5 .
score is -649 .

episode is 942 .
action_not_zero is 10 .
score is -4374 .

episode is 943 .
action_not_zero is 8 .
score is -893 .

episode is 944 .
action_not_zero is 9 .
score is -2483 .

episode is 945 .
action_not_zero is 13 .
score is -16177 .

episode is 946 .
action_not_zero is 5 .
score is -850 .

episode is 947 .
action_not_zero is 5 .
score is -643 .

episode is 948 .
action_not_zero is 16 .
score is -37487 .

episode is 949 .
action_not_zero is 19 .
score is -67554 .

episode is 950 .
action_not_zero is 10 .
score is -4358 .

episo

episode is 1073 .
action_not_zero is 7 .
score is -680 .

episode is 1074 .
action_not_zero is 13 .
score is -11107 .

episode is 1075 .
action_not_zero is 7 .
score is -859 .

episode is 1076 .
action_not_zero is 5 .
score is -828 .

episode is 1077 .
action_not_zero is 5 .
score is -675 .

episode is 1078 .
action_not_zero is 6 .
score is -696 .

episode is 1079 .
action_not_zero is 8 .
score is -874 .

episode is 1080 .
action_not_zero is 7 .
score is -620 .

episode is 1081 .
action_not_zero is 5 .
score is -807 .

episode is 1082 .
action_not_zero is 9 .
score is -2074 .

episode is 1083 .
action_not_zero is 10 .
score is -4120 .

episode is 1084 .
action_not_zero is 8 .
score is -1093 .

episode is 1085 .
action_not_zero is 5 .
score is -728 .

episode is 1086 .
action_not_zero is 9 .
score is -1878 .

episode is 1087 .
action_not_zero is 9 .
score is -754 .

episode is 1088 .
action_not_zero is 5 .
score is -1039 .

episode is 1089 .
action_not_zero is 6 .
score is -886 .

episo

episode is 1210 .
action_not_zero is 8 .
score is -1093 .

episode is 1211 .
action_not_zero is 12 .
score is -10906 .

episode is 1212 .
action_not_zero is 18 .
score is -55856 .

episode is 1213 .
action_not_zero is 18 .
score is -45883 .

episode is 1214 .
action_not_zero is 11 .
score is -6781 .

episode is 1215 .
action_not_zero is 8 .
score is -993 .

episode is 1216 .
action_not_zero is 14 .
score is -21973 .

episode is 1217 .
action_not_zero is 9 .
score is -1865 .

episode is 1218 .
action_not_zero is 11 .
score is -6837 .

episode is 1219 .
action_not_zero is 27 .
score is -190905 .

episode is 1220 .
action_not_zero is 32 .
score is -300907 .

episode is 1221 .
action_not_zero is 19 .
score is -66723 .

episode is 1222 .
action_not_zero is 14 .
score is -15929 .

episode is 1223 .
action_not_zero is 6 .
score is -947 .

episode is 1224 .
action_not_zero is 8 .
score is -1047 .

episode is 1225 .
action_not_zero is 32 .
score is -276660 .

episode is 1226 .
action_not_zero i

episode is 1345 .
action_not_zero is 32 .
score is -276874 .

episode is 1346 .
action_not_zero is 48 .
score is -820755 .

episode is 1347 .
action_not_zero is 47 .
score is -780748 .

episode is 1348 .
action_not_zero is 30 .
score is -254024 .

episode is 1349 .
action_not_zero is 36 .
score is -406854 .

episode is 1350 .
action_not_zero is 51 .
score is -903672 .

episode is 1351 .
action_not_zero is 39 .
score is -496778 .

episode is 1352 .
action_not_zero is 12 .
score is -10909 .

episode is 1353 .
action_not_zero is 7 .
score is -899 .

episode is 1354 .
action_not_zero is 9 .
score is -2018 .

episode is 1355 .
action_not_zero is 8 .
score is -933 .

episode is 1356 .
action_not_zero is 10 .
score is -3903 .

episode is 1357 .
action_not_zero is 11 .
score is -6919 .

episode is 1358 .
action_not_zero is 11 .
score is -6995 .

episode is 1359 .
action_not_zero is 15 .
score is -28946 .

episode is 1360 .
action_not_zero is 22 .
score is -105780 .

episode is 1361 .
action_no

episode is 1480 .
action_not_zero is 7 .
score is -799 .

episode is 1481 .
action_not_zero is 7 .
score is -899 .

episode is 1482 .
action_not_zero is 10 .
score is -3801 .

episode is 1483 .
action_not_zero is 11 .
score is -6883 .

episode is 1484 .
action_not_zero is 15 .
score is -29050 .

episode is 1485 .
action_not_zero is 4 .
score is -772 .

episode is 1486 .
action_not_zero is 9 .
score is -1798 .

episode is 1487 .
action_not_zero is 10 .
score is -3801 .

episode is 1488 .
action_not_zero is 8 .
score is -790 .

episode is 1489 .
action_not_zero is 6 .
score is -884 .

episode is 1490 .
action_not_zero is 6 .
score is -861 .

episode is 1491 .
action_not_zero is 5 .
score is -795 .

episode is 1492 .
action_not_zero is 5 .
score is -795 .

episode is 1493 .
action_not_zero is 9 .
score is -1924 .

episode is 1494 .
action_not_zero is 6 .
score is -797 .

episode is 1495 .
action_not_zero is 9 .
score is -1947 .

episode is 1496 .
action_not_zero is 9 .
score is -1989 .

e

### Given a constant input: reward = 19847

In [12]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(scores)
plt.yscale('symlog')
plt.title("Nitrogen Management")
plt.xlabel("Traing Steps")
plt.ylabel("Rewards")
plt.savefig("reward_0 water with exponential penalty learning_false.png")

In [12]:
plt.figure()
print(action_list)
plt.plot(action_list)
plt.title("Naction of last episode")
plt.xlabel("Days")
plt.ylabel("Nitrogen input")
plt.savefig("action_20 water with linear penalty learning_false.png")

[0, 0, 0, 0, 0, 0, 43, 0, 0, 0, 169, 0, 0, 0, 0, 0, 0, 0, 0, 78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 198, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 0, 0, 68, 0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 188, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 105, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100]


In [16]:
print(np.average(action_num_list))

8.52


In [28]:
score1=scores[-10:]
plt.plot(score1)
plt.title("Nitrogen Management")
plt.xlabel("Traing Steps")
plt.ylabel("Rewards")
plt.savefig("reward_partial_0 water with exponential penalty learning_false.png")

In [33]:
print(scores[-10:])

[-11112234.922243932, -1136.1068184592432, -1084.0878253100661, -996.9282381844116, -1112060.2553732428, -1112161.5164292145, -112175.58849308883, -1066.5072541128247, -2118.796046328218, -12061.030217774274]
